Required libraries

In [ ]:
%%capture
!pip install "langchain==0.2.7"
!pip install "langchain-core==0.2.20"
!pip install "langchain-text-splitters==0.2.2"
!pip install "lxml==5.2.2"
!pip install --user "langchain-community==0.2.1"

File download

In [ ]:
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/YRYau14UJyh0DdiLDdzFcA/companypolicies.txt"

--2025-03-23 17:34:50--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/YRYau14UJyh0DdiLDdzFcA/companypolicies.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15660 (15K) [text/plain]
Saving to: ‘companypolicies.txt’

companypolicies.txt 100%[===================>]  15.29K  --.-KB/s    in 0s      

2025-03-23 17:34:51 (52.4 MB/s) - ‘companypolicies.txt’ saved [15660/15660]



Loader
Splitter need string as input hence passing page_content of data (loader)

In [2]:
from langchain_community.document_loaders import TextLoader
import pprint
from pprint import pprint

In [3]:
loader = TextLoader("companypolicies.txt")
loader
data = loader.load()
pprint(data[0].page_content[:1000])

('1.\tCode of Conduct\n'
 '\n'
 'Our Code of Conduct outlines the fundamental principles and ethical '
 'standards that guide every member of our organization. We are committed to '
 'maintaining a workplace that is built on integrity, respect, and '
 'accountability.\n'
 'Integrity: We hold ourselves to the highest ethical standards. This means '
 'acting honestly and transparently in all our interactions, whether with '
 'colleagues, clients, or the broader community. We respect and protect '
 'sensitive information, and we avoid conflicts of interest.\n'
 "Respect: We embrace diversity and value each individual's contributions. "
 'Discrimination, harassment, or any form of disrespectful behavior is '
 'unacceptable. We create an inclusive environment where differences are '
 'celebrated and everyone is treated with dignity and courtesy.\n'
 'Accountability: We take responsibility for our actions and decisions. We '
 'follow all relevant laws and regulations, and we strive to contin

Split by Charachter

In [11]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)

texts_nometa = text_splitter.split_text(data[0].page_content)
texts_nometa

['1.\tCode of Conduct\n\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built',
 'kplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whe',
 'ur interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value e',
 "iversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrat",
 'erences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws 

In [12]:
#Number of chunks
len(texts_nometa)

87

In [15]:
texts = text_splitter.create_documents([data[0].page_content], metadatas=[{"document":"Company Policies"}])  # pass the metadata as well

In [24]:
texts[0]

Document(page_content='1.\tCode of Conduct')

Recursively Split by Character

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

texts = text_splitter.create_documents([data[0].page_content])
texts[0]

Document(page_content='1.\tCode of Conduct')

In [26]:
#number of chunks
len(texts)

215

Code Splitter : The CodeTextSplitter allows you to split your code, supporting multiple programming languages. 

In [27]:
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
[e.value for e in Language]

['cpp',
 'go',
 'java',
 'kotlin',
 'js',
 'ts',
 'php',
 'proto',
 'python',
 'rst',
 'ruby',
 'rust',
 'scala',
 'swift',
 'markdown',
 'latex',
 'html',
 'sol',
 'csharp',
 'cobol',
 'c',
 'lua',
 'perl',
 'haskell',
 'elixir']

In [29]:
#Python code splitter
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)
PYTHON_CODE = """
    def hello_world():
        print("Hello, World!")
    
    # Call the function
    hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(page_content='def hello_world():'),
 Document(page_content='print("Hello, World!")'),
 Document(page_content='# Call the function\n    hello_world()')]

In [30]:
#Java script code splitter
RecursiveCharacterTextSplitter.get_separators_for_language(Language.JS)
JS_CODE = """
    function helloWorld() {
      console.log("Hello, World!");
    }
    
    // Call the function
    helloWorld();
"""

js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS, chunk_size=60, chunk_overlap=0
)
js_docs = js_splitter.create_documents([JS_CODE])
js_docs

[Document(page_content='function helloWorld() {'),
 Document(page_content='console.log("Hello, World!");\n    }'),
 Document(page_content='// Call the function\n    helloWorld();')]

Mark down header splitter
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [31]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
md = "# Foo\n\n## Bar\n\nHi this is Jim\n\nHi this is Joe\n\n### Boo \n\nHi this is Lance \n\n## Baz\n\nHi this is Molly"

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(md)
md_header_splits

[Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}, page_content='Hi this is Jim  \nHi this is Joe'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar', 'Header 3': 'Boo'}, page_content='Hi this is Lance'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Baz'}, page_content='Hi this is Molly')]

In [32]:
# If header is needed in content, set strip_header=False
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(md)
md_header_splits

[Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar'}, page_content='# Foo  \n## Bar  \nHi this is Jim  \nHi this is Joe'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar', 'Header 3': 'Boo'}, page_content='### Boo  \nHi this is Lance'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Baz'}, page_content='## Baz  \nHi this is Molly')]

Split by HTML

In [34]:
#HTML text splitter
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
    <!DOCTYPE html>
    <html>
    <body>
        <div>
            <h1>Foo</h1>
            <p>Some intro text about Foo.</p>
            <div>
                <h2>Bar main section</h2>
                <p>Some intro text about Bar.</p>
                <h3>Bar subsection 1</h3>
                <p>Some text about the first subtopic of Bar.</p>
                <h3>Bar subsection 2</h3>
                <p>Some text about the second subtopic of Bar.</p>
            </div>
            <div>
                <h2>Baz</h2>
                <p>Some text about Baz</p>
            </div>
            <br>
            <p>Some concluding text about Foo</p>
        </div>
    </body>
    </html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

[Document(page_content='Foo'),
 Document(metadata={'Header 1': 'Foo'}, page_content='Some intro text about Foo.  \nBar main section Bar subsection 1 Bar subsection 2'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section'}, page_content='Some intro text about Bar.'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 1'}, page_content='Some text about the first subtopic of Bar.'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 2'}, page_content='Some text about the second subtopic of Bar.'),
 Document(metadata={'Header 1': 'Foo'}, page_content='Baz'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Baz'}, page_content='Some text about Baz'),
 Document(metadata={'Header 1': 'Foo'}, page_content='Some concluding text about Foo')]

In [36]:
#section splitter for HTML. HTMLSectionSplitter is also a "structure-aware" chunker that splits text section by section based on headings.
from langchain_text_splitters import HTMLSectionSplitter
html_splitter = HTMLSectionSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

[Document(metadata={'Header 1': 'Foo'}, page_content='Foo \n Some intro text about Foo.'),
 Document(metadata={'Header 2': 'Bar main section'}, page_content='Bar main section \n Some intro text about Bar.'),
 Document(metadata={'Header 3': 'Bar subsection 1'}, page_content='Bar subsection 1 \n Some text about the first subtopic of Bar.'),
 Document(metadata={'Header 3': 'Bar subsection 2'}, page_content='Bar subsection 2 \n Some text about the second subtopic of Bar.'),
 Document(metadata={'Header 2': 'Baz'}, page_content='Baz \n Some text about Baz \n \n \n Some concluding text about Foo')]